In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import sys
sys.path.append(r'/home/gabriel/Documents/proj/utils/')
from utils import createSparkSesion
from pyspark.sql import (functions as F,
                                        Window)
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#nao esquecer , pra criar um df spark a funcao spark.createDataFrame([lista de tuplas com os dados],schema=schema em structType)
# row number precisa ser ordenada por algo
# rank pula a posicao em caso de empate dense rank segue a contagem , rank ex(1,2,2,4) dense rank ex(1,2,2,3)
# nulos , dropar , colocar um valor padrao , pra strings ou valores inteiros ,podendo ser um valor calculado dependendo da regra de negocio
'''joined_df = df1.join(
    df2,
    (df1.id == df2.user_id) & (df1.date == df2.event_date),
    how="inner"
)

Nao esquecer join com mais de uma chave'''

In [3]:
spark = createSparkSesion()

25/05/20 22:00:03 WARN Utils: Your hostname, GabrielUnbunto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/20 22:00:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/20 22:00:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark

In [5]:
path = "/home/gabriel/Documents/local_datasets/raw/archive/athletes.csv"

In [6]:
schema = StructType([
    StructField("id",               IntegerType(),  nullable=True),
    StructField("name",             StringType(),   nullable=True),
    StructField("gender",           StringType(),   nullable=True),
    StructField("age",              IntegerType(),  nullable=True),
    StructField("height",           IntegerType(),  nullable=True),
    StructField("weight",           IntegerType(),  nullable=True),
    StructField("team",             StringType(),   nullable=True),
    StructField("noc",              StringType(),   nullable=True),
    StructField("games",            StringType(),   nullable=True),
    StructField("year",             IntegerType(),  nullable=True),
    StructField("season",           StringType(),   nullable=True),
    StructField("city",             StringType(),   nullable=True),
    StructField("sport",            StringType(),   nullable=True),
    StructField("event",            StringType(),   nullable=True),
    StructField("medal",            StringType(),   nullable=True)
])

In [7]:
competitios_history = spark.read.csv(path , header = True , schema = schema)

In [8]:
competitios_history.takePandas()

,id,name,gender,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,1,A Dijiang,M,24,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NA
1,2,A Lamusi,M,23,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NA
2,3,Gunnar Nielsen Aaby,M,24,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NA
3,4,Edgar Lindenau Aabye,M,34,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NA


In [9]:
competitios_history.count_nulls()

,id,name,gender,age,height,weight,team,noc,games,year,season,city,sport,event,medal
0,0,0,0,9474,60171,63933,0,0,0,0,0,0,0,0,0


distribuicao de medalhas por times , por siglas ,por idade , agrupado ou nao por ano ,

In [ ]:
gender_progression_over_year = (
   competitios_history
      .withColumn("gender", F.when(F.col("gender") == 'M', 'Male').otherwise('Female'))
      .groupBy("gender","year")
      .agg(
         F.count("gender")
      )
      .orderBy(F.asc("year")))

In [27]:
#doing better

#count de M no ano e F no ano , soma e divisao pra ter a %
#resultado esperado 'e  , ano M count , M share e vise versa

gender_progression_over_year_2 = (
   competitios_history
      .withColumn("gender", F.when(F.col("gender") == 'M', 'Male').otherwise('Female'))
      .groupBy("year")
      .agg(

          F.count(F.when(F.col("gender")== "Male",True)).alias("male_count"),
         (F.count(F.when(F.col("gender")== "Male",True)) / F.count("gender")).alias("male_share"),

          F.count(F.when(F.col("gender")== "Female",True)).alias("female_count"),
         (F.count(F.when(F.col("gender")== "Female",True)) / F.count("gender")).alias("female_share"),

          F.count("gender").alias("total")
      )
      .orderBy(F.asc("year")))

In [28]:
gender_progression_over_year_2.takePandas()

,year,male_count,male_share,female_count,female_share,total
0,1896,380,1.000000,0,0.000000,380
1,1900,1903,0.982955,33,0.017045,1936
2,1904,1285,0.987702,16,0.012298,1301
3,1906,1722,0.993653,11,0.006347,1733
4,1908,3054,0.984844,47,0.015156,3101


In [36]:
window_female_progression = Window.orderBy("year")#i believe that ascending ord is default 
female_progression_year_over_year = (
   gender_progression_over_year_2
      .select("year","female_count")
      .withColumn("past_year_count" , F.lag("female_count").over(window_female_progression))
      .withColumn("yoy",  F.col("female_count") / F.col("past_year_count"))
)



In [38]:
female_progression_year_over_year.takePandas(20)

25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 2

,year,female_count,past_year_count,yoy
0,1896,0,NaN,NaN
1,1900,33,0.0,NaN
2,1904,16,33.0,0.484848
3,1906,11,16.0,0.687500
4,1908,47,11.0,4.272727
5,1912,87,47.0,1.851064
6,1920,134,87.0,1.540230
7,1924,261,134.0,1.947761
8,1928,437,261.0,1.674330
9,1932,369,437.0,0.844394


In [40]:
female_progression_year_over_year.select(F.avg("yoy")).show()

25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 22:39:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/20 2

+------------------+
|          avg(yoy)|
+------------------+
|1.6103394339242676|
+------------------+

